# Introduction, motivation, problem statement

Clearly define the problem statemend or purpose of the project


# Data sources or RL task

Data sources or reinforcement learning tasks are clearly documented and described

# Exploratory Analysis of Data or RL tasks

Proide details about the properties, number of classes, pre-processing, challenging aspects, etc of the data.

Describe the environment.

# Models and/or method

Models and methods are judiciously chosen and appropriately applied. If building on previous work, identify the source and clearly delineate which parts are your own work.

# Results

# Baseline

### Environment

In [ ]:
!pip install airsim

  Using cached airsim-1.8.1.tar.gz (20 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import airsim

import csv
import math
import pprint
import time
from PIL import Image

import numpy as np

class DroneEnv(object):
    """Drone environment class using AirSim python API"""

    def __init__(self):
        self.client = airsim.MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)

        self.pose = self.client.simGetVehiclePose()
        self.state = self.client.getMultirotorState().kinematics_estimated.position
        print(self.state.x_val, self.state.y_val, self.state.z_val)
        self.quad_offset = (0, 0, 0)
        initX = 162
        initY = -320
        initZ = -150

        self.start_collision = "Cube"
        self.next_collision = "Cube"
        self.cnt_collision = 0
        self.collision_change = False

        self.client.takeoffAsync().join()
        print("take off moving positon")
        self.client.moveToPositionAsync(initX, initY, initZ, 5).join()

        self.ep = 0

    def step(self, action):
        """Step"""
        print("doing step")
        self.quad_offset = self.interpret_action(action)
        print("quad_offset: ", self.quad_offset)
        quad_state = self.client.getMultirotorState().kinematics_estimated.position
        quad_vel = self.client.getMultirotorState().kinematics_estimated.linear_velocity
        self.client.moveByVelocityAsync(
            quad_vel.x_val + self.quad_offset[0],
            quad_vel.y_val + self.quad_offset[1],
            quad_vel.z_val + self.quad_offset[2],
            20,
        ).join()
        time.sleep(0.5)

        collision_info = self.client.simGetCollisionInfo()

        if self.next_collision != collision_info.object_name:
            self.collision_change = True

        if collision_info.has_collided:
            if self.cnt_collision == 0:
                self.start_collision = collision_info.object_name
                self.next_collision = collision_info.object_name
                self.cnt_collision = 1
            else:
                self.next_collision = collision_info.object_name

        quad_state = self.client.getMultirotorState().kinematics_estimated.position
        quad_vel = self.client.getMultirotorState().kinematics_estimated.linear_velocity
        print(
            "state x:",
            quad_state.x_val,
            " y: ",
            quad_state.y_val,
            " z: ",
            quad_state.z_val,
        )

        result = self.compute_reward(quad_state, quad_vel, collision_info)
        state = self.get_obs()
        done = self.isDone(result)
        return state, result, done

    def reset(self):
        """Reset to initial state"""
        self.client = airsim.MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)

        self.pose = self.client.simGetVehiclePose()
        self.state = self.client.getMultirotorState().kinematics_estimated.position
        print(self.state.x_val, self.state.y_val, self.state.z_val)
        self.quad_offset = (0, 0, 0)
        initX = 162
        initY = -320
        initZ = -150

        self.start_collision = "Cube"
        self.next_collision = "Cube"
        self.cnt_collision = 0
        self.collision_change = False

        self.client.takeoffAsync().join()
        print("take off moving positon")
        self.client.moveToPositionAsync(initX, initY, initZ, 5).join()
        responses = self.client.simGetImages(
            [airsim.ImageRequest("1", airsim.ImageType.Scene, False, False)]
        )
        obs = self.transform_input(responses)

        return obs

    def get_obs(self):
        """Get observation"""
        responses = self.client.simGetImages(
            [airsim.ImageRequest("1", airsim.ImageType.Scene, False, False)]
        )
        obs = self.transform_input(responses)
        return obs

    def get_distance(self, quad_state):
        """Get distance between current state and goal state"""
        pts = np.array([-10, 10, -10])
        quad_pt = np.array(list((quad_state.x_val, quad_state.y_val, quad_state.z_val)))
        dist = np.linalg.norm(quad_pt - pts)
        return dist

    def compute_reward(self, quad_state, quad_vel, collision_info):
        """Compute reward"""
        thresh_dist = 7
        max_dist = 500
        beta = 1

        z = -10
        if self.ep == 0:
            if (
                self.collision_change == True
                and self.next_collision != self.start_collision
            ):
                if "Cube" in self.next_collision:
                    dist = 10000000
                    dist = self.get_distance(quad_state)
                    reward = 50000
                else:
                    reward = -100
            else:
                reward = 0
        else:
            if self.next_collision != self.start_collision:
                if "Cube" in self.next_collision:
                    dist = 10000000
                    dist = self.get_distance(quad_state)
                    reward = 50000
                else:
                    reward = -100
            else:
                reward = 0
        if quad_state.z_val < -280:
            reward = -100
        print(reward)
        return reward


    def isDone(self, reward):
        """Check if episode is done"""
        done = 0
        if reward <= -10:
            done = 1
            self.client.armDisarm(False)
            self.client.reset()
            self.client.enableApiControl(False)
            time.sleep(1)
        elif reward > 499:
            done = 1
            self.client.armDisarm(False)
            self.client.reset()
            self.client.enableApiControl(False)
            time.sleep(1)
        return done

    def transform_input(self, responses):
        """Transform input binary array to image"""
        response = responses[0]
        img1d = np.fromstring(
            response.image_data_uint8, dtype=np.uint8
        )
        img_rgba = img1d.reshape(
            response.height, response.width, 4
        )
        img2d = np.flipud(img_rgba)

        image = Image.fromarray(img2d)
        im_final = np.array(image.resize((84, 84)).convert("L"))

        return im_final

    def interpret_action(self, action):
        """Interprete action"""
        scaling_factor = 5
        if action.item() == 0:
            self.quad_offset = (0, 0, 0)
        elif action.item() == 1:
            self.quad_offset = (scaling_factor, 0, 0)
        elif action.item() == 2:
            self.quad_offset = (0, scaling_factor, 0)
        elif action.item() == 3:
            self.quad_offset = (0, 0, scaling_factor)
        elif action.item() == 4:
            self.quad_offset = (-scaling_factor, 0, 0)
        elif action.item() == 5:
            self.quad_offset = (0, -scaling_factor, 0)
        elif action.item() == 6:
            self.quad_offset = (0, 0, -scaling_factor)

        return self.quad_offset

ModuleNotFoundError: No module named 'airsim'

### Agent (simple)

In [ ]:
import gym
from gym import wrappers
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import deque
import numpy as np

from env import DroneEnv


env = DroneEnv()


EPISODES = 50  # number of episodes
EPS_START = 0.9  # e-greedy threshold start value
EPS_END = 0.05  # e-greedy threshold end value
EPS_DECAY = 200  # e-greedy threshold decay
GAMMA = 0.8  # Q-learning discount factor
LR = 0.001  # NN optimizer learning rate
BATCH_SIZE = 1  # Q-learning batch size


class DQNAgent:
    def __init__(self):
        self.model = nn.Sequential(
            nn.Linear(84, 21),
            nn.ReLU(),
            nn.Linear(21, 7)
        )
        self.memory = deque(maxlen=10000)
        self.optimizer = optim.Adam(self.model.parameters(), LR)
        self.steps_done = 0

    def act(self, state):
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * self.steps_done / EPS_DECAY)
        self.steps_done += 1
        if random.random() > eps_threshold:
            action = self.model(state).data.max(1)[1]
            action = [action.max(1)[1]]
            return torch.LongTensor([action])
        else:
            action = [random.randrange(0, 7)]
            return torch.LongTensor([action])

    def memorize(self, state, action, reward, next_state):
        self.memory.append((state,
                            action,
                            torch.FloatTensor([reward]),
                            torch.FloatTensor([next_state])))

    def learn(self):
        """Experience Replay"""
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        states, actions, rewards, next_states = zip(*batch)
        states = torch.cat(states)
        actions = torch.cat(actions)
        rewards = torch.cat(rewards)
        next_states = torch.cat(next_states)

        current_q = self.model(states)
        max_next_q = self.model(next_states).detach().max(1)[0]
        expected_q = rewards + (GAMMA * max_next_q)

        loss = F.mse_loss(current_q.squeeze(), expected_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


agent = DQNAgent()
score_history = []
reward_history = []
score = 0

for e in range(1, EPISODES+1):
    state = env.reset()
    steps = 0
    while True:
        state = torch.FloatTensor([state])
        action = agent.act(state)
        next_state, reward, done = env.step(action)

        agent.memorize(state, action, reward, next_state)
        agent.learn()

        state = next_state
        steps += 1
        score += reward

        if done:
            print("episode:{0}, reward: {1}, score: {2}".format(e, reward, score))
            print("----------------------------------------------------")
            score_history.append(steps)
            reward_history.append(reward)
            f = open('reward.txt', 'a')
            f.write(str(reward))
            f.close()
            f2 = open('score.txt', 'a')
            f2.write(str(score))
            f2.close()
            break

ModuleNotFoundError: No module named 'env'

### Agent (more complex)

In [ ]:
import math
import random
from collections import deque

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import gym
from gym import wrappers
from env import DroneEnv

env = DroneEnv()


# Deep Q-network with arbitrary values
class DQN(nn.Module):
    def __init__(self, in_channels=84, num_actions=7):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 84, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(84, 42, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(42, 21, kernel_size=3, stride=1)
        self.fc4 = nn.Linear(336, 168)
        self.fc5 = nn.Linear(168, num_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc4(x.view(x.size(0), -1)))
        return self.fc5(x)


class Agent:
    def __init__(
        self,
        eps_start=0.9,
        eps_end=0.05,
        eps_decay=200,
        gamma=0.8,
        learning_rate=0.001,
        batch_size=1,
    ):
        self.eps_start = eps_start
        self.eps_end = eps_end
        self.eps_decay = eps_decay
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        dqn = DQN()
        self.model = dqn.forward()
        self.memory = deque(maxlen=10000)
        self.optimizer = optim.Adam(self.model.parameters(), self.learning_rate)
        self.steps_done = 0

    def act(self, state):
        eps_threshold = self.eps_end + (self.eps_start - self.eps_end) * math.exp(
            -1.0 * self.steps_done / self.eps_decay
        )
        self.steps_done += 1
        if random.random() > eps_threshold:
            action = self.model(state).data.max(1)[1]
            action = [action.max(1)[1]]
            return torch.LongTensor([action])
        else:
            action = [random.randrange(0, 7)]
            return torch.LongTensor([action])

    def memorize(self, state, action, reward, next_state):
        self.memory.append(
            (
                state,
                action,
                torch.FloatTensor([reward]),
                torch.FloatTensor([next_state]),
            )
        )

    def learn(self):
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states = zip(*batch)
        print(actions)
        states = torch.cat(states)
        actions = torch.cat(actions)
        rewards = torch.cat(rewards)
        next_states = torch.cat(next_states)
        print(actions)
        current_q = self.model(states)
        max_next_q = self.model(next_states).detach().max(1)[0]
        expected_q = rewards + (GAMMA * max_next_q)

        loss = F.mse_loss(current_q.squeeze(), expected_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def train(self):
        score_history = []
        reward_history = []
        score = 0

        for e in range(1, EPISODES + 1):
        state = env.reset()
        steps = 0
        while True:
            state = torch.FloatTensor([state])
            action = act(state)
            print(action)
            next_state, reward, done = env.step(action)

            memorize(state, action, reward, next_state)
            learn()

            state = next_state
            steps += 1
            score += reward

            if done:
                print("episode:{0}, reward: {1}, score: {2}".format(e, reward, score))
                print("----------------------------------------------------")
                score_history.append(steps)
                reward_history.append(reward)
                f = open("reward.txt", "a")
                f.write(str(reward))
                f.close()
                f2 = open("score.txt", "a")
                f2.write(str(score))
                f2.close()
                break

IndentationError: expected an indented block after 'for' statement on line 109 (<ipython-input-3-9fc886aed666>, line 110)

### Run the program



In [ ]:
agent = Agent()
agent.train()